In [1]:
import json
import re
import random
import copy

In [3]:
domain= "Tabletop"

In [2]:
def make_dataset(prompt_out_steps_list, high_insts, used_objs):
    
    used_objs_list= list()
    steps_list= list()
    high_inst_list= list()
    noise= list()
    _dataset= prompt_out_steps_list
    
    dataset= dict()
    for idx, _data in enumerate(_dataset):
        
        _data= re.sub(r'\n{2,}', '\n', _data)
        steps= _data.strip().split(sep='=', maxsplit=1)
        
        if len(steps) <= 1:
            steps= _data.strip().split(sep=':', maxsplit=1)
            if len(steps) <= 1:
                print("There is noise :", idx, '\n', _data)
                print('#'*20, end='\n\n')
                noise.append(idx)
                continue
        
        steps= steps[1].strip()
        steps= steps.split("\n")
        steps= [step.strip() for step in steps]
        
        steps_list.append(steps)
        used_objs_list.append(used_objs[idx])
        high_inst_list.append(high_insts[idx])
    
    dataset['steps']= steps_list
    dataset['used_objects']= used_objs_list
    dataset['high_instructions']= high_inst_list
    
    return dataset, noise

In [4]:
file_path= "./datasets/{}/_Dataset.json".format(domain)
with open(file_path, 'r') as json_file:
    _dataset= json.load(json_file)
    
print(_dataset.keys())

In [6]:
file_path= "./datasets/{}/Commands.json".format(domain)
with open(file_path, 'r') as json_file:
    _commands= json.load(json_file)
    
print(_commands.keys())

In [8]:
_dataset['used_objects']= copy.deepcopy(_commands['used_objects'])

In [10]:
dataset, noise = make_dataset(_dataset['prompt_output(steps)'], _dataset['high_instructions'], _dataset['used_objects'])

In [12]:
_dataset['objects']= list(set(_dataset['objects']))
dataset['objects']=list()

# Capitalize the first character
for obj in _dataset['objects']:
    obj= obj[0].upper() + obj[1:]
    dataset['objects'].append(obj.strip())

In [44]:
proposed_objs_list= list()
_used_obj_list= copy.deepcopy(dataset['used_objects'])

# Remove duplicate objects
for idx, objs in enumerate(_used_obj_list):
    _used_obj_list[idx]= list(set(_used_obj_list[idx]))
    
    for i, _ in enumerate(_used_obj_list[idx]):
        _used_obj_list[idx][i]= _used_obj_list[idx][i].lower()
        
        if len(_used_obj_list[idx][i].strip()) <= 1:
            print(idx, i, _used_obj_list[idx][i])
            _used_obj_list[idx].remove(_used_obj_list[idx][i])
            continue

# Make final object list : required objects to execute commands + random objects
for used_obj in _used_obj_list:
    random_picked_objs= list()
    filtered_list= list()
    
    for item in dataset['objects']:
        if item.lower() not in used_obj:
            filtered_list.append(item)
            
    random_picked_objs = random.sample(filtered_list, random.choice([1, 3]))
    used_obj= [obj[0].upper()+obj[1:] for obj in used_obj]
    
    proposed_objs= copy.deepcopy(used_obj)
    proposed_objs.extend(random_picked_objs)
    random.shuffle(proposed_objs)
    proposed_objs_list.append(proposed_objs)

In [46]:
# Check if there are duplicate objects, and remove duplicate objects.
for obj_list in _used_obj_list:
    _obj_list= list(set(obj_list))
    if len(_obj_list) != len(obj_list):
        print(obj_list)
        print(_obj_list)

In [47]:
dataset['objects_on_table']= copy.deepcopy(proposed_objs_list)
dataset['place']= _dataset['place']

print(dataset.keys())

In [50]:
file_path= "./datasets/{}/".format(domain)

with open(file_path+'Dataset.json', 'w') as json_file:
    json.dump(dataset, json_file, indent='\t')